## Gai Dialogue System

This system is responsible for managing and storing the dialogue messages between the user and the agents. It also handles the context and history of the conversation. It has a dependency on `Gai Messages System`.


---

## 1. Dialogue Bus

### a) Setup

In [ ]:
from gai.dialogue import DialogueBus
from gai.dialogue.nodes import AgentNode, UserNode
from rich import print as rprint

class MockAgentNode(AgentNode):
    
    def __init__(self, name: str, dialogue: DialogueBus):
        super().__init__(name=name, dialogue=dialogue, handle_send_cb=self.handle_send_cb)
        
    def handle_send_cb(self, message):
        
        # Simulate a response from the agent
        
        if self.name == "Sara":
            message = "This is a story about a brave knight who fought a dragon.".split()
            for word in message:
                yield word+" "
        if self.name == "Diana":
            message = "The knight was very brave and fought valiantly.".split()
            for word in message:
                yield word+" "

# 1. Initiate a dialogue session

dialogue = DialogueBus()
await dialogue.start()

# 2. Add 1 User and 2 Agents to the Dialogue
    
sara = MockAgentNode(name="Sara", dialogue=dialogue)
diana = MockAgentNode(name="Diana", dialogue=dialogue)

async def handle_reply_cb(message):
    rprint(f"[bold green]{message.header.sender} replied:[/bold green] {message.body.content}")

user = UserNode(dialogue=dialogue, handle_reply_cb=handle_reply_cb)


/workspaces/gai-sdk/src/gai/dialogue/nodes/user_node.py:46: RuntimeWarning: coroutine 'handle_reply_cb' was never awaited
  self.handle_reply_cb(message)


### b) Send and Reply

In [2]:
# 3. Ask Sara to tell a story

await user.chat("Tell me a one paragraph story","Sara") 
rprint("[User] [bold bright_yellow] Sent message to Sara [/bold bright_yellow]")

# 4. Ask Diana to continue   

await user.chat("Please continue","Diana")
rprint("[User] [bold bright_yellow] Sent message to Diana [/bold bright_yellow]")



[User]  Sent message to Sara 

[User]  Sent message to Diana 

[Sara] Handle send message. message=id='3335df7a-64a5-4eb9-a999-348411e0615d' 
header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1749748694.139189, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=1, 
message_id='00000000-0000-0000-0000-000000000000.1', content_type='text', content='Tell me a one paragraph story')

[Diana] Message not for me, ignore. message=id='3335df7a-64a5-4eb9-a999-348411e0615d' 
header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1749748694.139189, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=1, 
message_id='00000000-0000-0000-0000-000000000000.1', content_type='text', content='Tell me a one paragraph story')

[Sara] Message not for me, ignore. message=id='43e8ab94-4bcf-47e6-b48b-52864c0f578c' 
header=MessageHeaderPydantic(sender='User', recipient='Diana', timestamp=1749748694.1496444, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=2, 
message_id='00000000-0000-0000-0000-000000000000.2', content_type='text', content='Please continue')

[Diana] Handle send message. message=id='43e8ab94-4bcf-47e6-b48b-52864c0f578c' 
header=MessageHeaderPydantic(sender='User', recipient='Diana', timestamp=1749748694.1496444, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=2, 
message_id='00000000-0000-0000-0000-000000000000.2', content_type='text', content='Please continue')

### c) Send broadcast messages

In [3]:
# 1. Send a broadcast message to all agents

await user.chat("Introduce yourselves","*") 
print("[User] [bold bright_yellow] Sent broadcast message [/bold bright_yellow]")



[User] [bold bright_yellow] Sent broadcast message [/bold bright_yellow]


[Sara] Handle broadcast message. message=id='305e771e-23ef-4af4-9e5a-8f5f830a2f9a' 
header=MessageHeaderPydantic(sender='User', recipient='*', timestamp=1749573912.1055892, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=25, 
message_id='00000000-0000-0000-0000-000000000000.25', content_type='text', content='Introduce yourselves')

[Diana] Handle broadcast message. message=id='305e771e-23ef-4af4-9e5a-8f5f830a2f9a' 
header=MessageHeaderPydantic(sender='User', recipient='*', timestamp=1749573912.1055892, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=25, 
message_id='00000000-0000-0000-0000-000000000000.25', content_type='text', content='Introduce yourselves')

In [4]:
print(dialogue.list_messages())

[MessagePydantic(id='6434de5f-7823-4308-9c83-c279d3f0d37d', header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1749573909.008059, order=0), body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=1, message_id='00000000-0000-0000-0000-000000000000.1', content_type='text', content='Tell me a one paragraph story')), MessagePydantic(id='72ca8b42-8420-466e-b684-7a22e5f38f25', header=MessageHeaderPydantic(sender='User', recipient='Diana', timestamp=1749573909.0158813, order=0), body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=2, message_id='00000000-0000-0000-0000-000000000000.2', content_type='text', content='Please continue')), MessagePydantic(id='832e8f01-8694-4f54-9293-eb93e1d59eee', header=MessageHeaderPydantic(sender='Sara', recipient='User', timestamp=1749573909.0288253, order=0), body=ReplyBodyPydantic(type='reply', dialogue_id='00000000-0000-0000-0000-000000000000', messa

---

## 4. File Dialogue Bus

In [ ]:
import asyncio
from gai.messages import message_helper
from gai.dialogue import FileDialogueBus, DialogueBus
from rich import print

from gai.dialogue.nodes import AgentNode, UserNode

class MockAgentNode(AgentNode):
    
    def __init__(self, name: str, dialogue: DialogueBus):
        super().__init__(name=name, dialogue=dialogue)
        
    def _stream_completion(self, message):
        
        # Simulate a response from the agent
        
        if self.name == "Sara":
            yield "This is a story about a brave knight who fought a dragon."
        if self.name == "Diana":
            yield "The knight was very brave and fought valiantly."

# 1. Initiate a dialogue session
from gai.lib.tests import make_local_tmp
here = make_local_tmp()
dialogue = FileDialogueBus(logger_name="User", app_dir=here,reset=True)
await dialogue.start()

# 2. Add 1 User and 2 Agents to the Dialogue
    
sara = MockAgentNode(name="Sara", dialogue=dialogue)
diana = MockAgentNode(name="Diana", dialogue=dialogue)
user = UserNode(dialogue=dialogue)


Task was destroyed but it is pending!
task: <Task pending name='Task-6' coro=<AsyncMessageBus._dispatch_loop() running at /workspaces/gai-sdk/messages/src/gai/messages/async_message_bus.py:295> wait_for=<Future pending cb=[Task.task_wakeup()]>>


### b) Send and Reply

In [6]:
# 3. Ask Sara to tell a story

await user.chat("Tell me a one paragraph story","Sara") 
print("[User] [bold bright_yellow] Sent message to Sara [/bold bright_yellow]")

# # 4. Ask Diana to continue   

await user.chat("Please continue","Diana")
print("[User] [bold bright_yellow] Sent message to Diana [/bold bright_yellow]")



[User]  Sent message to Sara 

[User]  Sent message to Diana 

[Sara] Handle send message. message=id='eaea9bd5-4216-4682-a2d7-9ea808923053' 
header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1749569449.2860677, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=48, 
message_id='00000000-0000-0000-0000-000000000000.48', content_type='text', content='Tell me a one paragraph story')

[Diana] Message not for me, ignore. message=id='eaea9bd5-4216-4682-a2d7-9ea808923053' 
header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1749569449.2860677, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=48, 
message_id='00000000-0000-0000-0000-000000000000.48', content_type='text', content='Tell me a one paragraph story')

[Sara] Message not for me, ignore. message=id='34cfa1d7-42f2-4952-9c3d-044c2ed45882' 
header=MessageHeaderPydantic(sender='User', recipient='Diana', timestamp=1749569449.2906847, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=49, 
message_id='00000000-0000-0000-0000-000000000000.49', content_type='text', content='Please continue')

[Diana] Handle send message. message=id='34cfa1d7-42f2-4952-9c3d-044c2ed45882' 
header=MessageHeaderPydantic(sender='User', recipient='Diana', timestamp=1749569449.2906847, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=49, 
message_id='00000000-0000-0000-0000-000000000000.49', content_type='text', content='Please continue')

### c) Send broadcast messages

In [7]:
# 1. Send a broadcast message to all agents

await user.chat("Introduce yourselves","*") 
print("[User] [bold bright_yellow] Sent broadcast message [/bold bright_yellow]")



[User]  Sent broadcast message 

[Sara] Handle broadcast message. message=id='b63b1aad-f190-48d2-87de-e3a9a4946b50' 
header=MessageHeaderPydantic(sender='User', recipient='*', timestamp=1749569451.8893034, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=51, 
message_id='00000000-0000-0000-0000-000000000000.51', content_type='text', content='Introduce yourselves')

[Diana] Handle broadcast message. message=id='b63b1aad-f190-48d2-87de-e3a9a4946b50' 
header=MessageHeaderPydantic(sender='User', recipient='*', timestamp=1749569451.8893034, order=0) 
body=SendBodyPydantic(type='send', dialogue_id='00000000-0000-0000-0000-000000000000', message_no=51, 
message_id='00000000-0000-0000-0000-000000000000.51', content_type='text', content='Introduce yourselves')